In [1]:
# Import libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
def find_news(tickers):
    finwiz_url = 'https://finviz.com/quote.ashx?t='
    news_tables = {}
    for ticker in tickers:
        url = finwiz_url + ticker
        req = Request(url=url, headers = {'User-Agent': 'Mozilla/5.0'})
        response = urlopen(req)    
        # Read the contents of the file into 'html'
        html = BeautifulSoup(response)
        # Find 'news-table' in the Soup and load it into 'news_table'
        news_table = html.find(id='news-table')
        # Add the table to our dictionary
        news_tables[ticker] = news_table
        
    # Read one single day of headlines for 'AMZN' 
    amzn = news_tables['AMZN']

    # Get all the table rows tagged in HTML with <tr> into 'amzn_tr'
    amzn_tr = amzn.findAll('tr')

    for i, table_row in enumerate(amzn_tr):
        # Read the text of the element 'a' into 'link_text'
        a_text = table_row.a.text
        # Read the text of the element 'td' into 'data_text'
        td_text = table_row.td.text
        # Print the contents of 'link_text' and 'data_text' 
      #  print(a_text)
     #   print(td_text)
        # Exit after printing 4 rows of data
        if i == 5000:
            break
    parsed_news = []

    # Iterate through the news
    for file_name, news_table in news_tables.items():
        # Iterate through all tr tags in 'news_table'
        for x in news_table.findAll('tr'):
            # read the text from each tr tag into text
            # get text from a only
            text = x.a.get_text() 
            # splite text in the td tag into a list 
            date_scrape = x.td.text.split()
            # if the length of 'date_scrape' is 1, load 'time' as the only element

            if len(date_scrape) == 1:
                time = date_scrape[0]
            
            # else load 'date' as the 1st element and 'time' as the second    
            else:
                date = date_scrape[0]
                time = date_scrape[1]
            # Extract the ticker from the file name, get the string up to the 1st '_'  
            ticker = file_name.split('_')[0]
        
            # Append ticker, date, time and headline as a list to the 'parsed_news' list
            parsed_news.append([ticker, date, time, text])
    
        # Instantiate the sentiment intensity analyzer
    vader = SentimentIntensityAnalyzer()

    # Set column names
    columns = ['ticker', 'date', 'time', 'headline']

    # Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
    parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)

    # Iterate through the headlines and get the polarity scores using vader
    scores = parsed_and_scored_news['headline'].apply(vader.polarity_scores).tolist()

    # Convert the 'scores' list of dicts into a DataFrame
    scores_df = pd.DataFrame(scores)

    # Join the DataFrames of the news and the list of dicts
    parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')

    # Convert the date column from string to datetime
    parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date
    
    plt.rcParams['figure.figsize'] = [10, 6]

    # Group by date and ticker columns from scored_news and calculate the mean
    mean_scores = parsed_and_scored_news.groupby(['ticker','date']).mean()

    # Unstack the column ticker
    mean_scores = mean_scores.unstack()

    # Get the cross-section of compound in the 'columns' axis
    mean_scores = mean_scores.xs('compound', axis="columns").transpose()
    
    return mean_scores
        


In [3]:
tickers_df = pd.read_csv("sandp500.csv")
tickers_list = tickers_df["Ticker"].tolist()
tickers_list[0] = "AMZN"
find_news(tickers_list[:5])

ticker,AMZN,FB,MSFT,TSLA
date,,,,
2021-10-04,NaN,NaN,0.011411,NaN
2021-10-05,NaN,NaN,0.147686,NaN
2021-10-06,0.138192,-0.082873,0.154024,NaN
2021-10-07,0.084460,-0.108231,0.120425,0.047263
2021-10-08,0.086078,0.131254,0.086670,0.070088
2021-10-09,0.187640,0.248840,0.499050,0.049727
2021-10-10,0.190900,0.106200,NaN,0.068233
2021-10-11,0.286065,-0.048425,0.162817,0.109946


In [4]:
df1 = find_news(tickers_list[:5])
df1 = df1.replace(np.nan, 0)
tickers_list[4] = "AMZN"
df2 = find_news(tickers_list[4:7])
df2 = df2.replace(np.nan, 0)
tickers_list[7] = "AMZN"
df3 = find_news(tickers_list[7:198])
df3 = df3.replace(np.nan, 0)

In [5]:
tickers_list[198] = "AMZN"
df4 = find_news(tickers_list[198:253])
df4 = df4.replace(np.nan, 0)
df4

ticker,ADM,AFL,ALL,AME,AMZN,ANSS,AWK,AZO,BLL,CARR,...,TDG,VFC,VRSK,WBA,WEC,WELL,WLTW,WMB,YUM,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2020-06-12,0.0000,0.000000,0.0,0.0,0.000000,0.0000,0.0000,0.00,0.0,0.0000,...,0.0,0.0,0.0000,0.00000,0.0000,0.0000,0.00000,0.0000,0.0,0.0
2020-06-15,0.0000,0.000000,0.0,0.0,0.000000,0.0000,0.0000,0.00,0.0,0.0000,...,0.0,0.0,0.0000,0.00000,0.0000,0.0000,0.00000,0.0000,0.0,0.0
2020-06-16,0.0000,0.000000,0.0,0.0,0.000000,0.0000,0.0000,0.00,0.0,0.0000,...,0.0,0.0,0.0000,0.00000,0.0000,0.0000,0.00000,0.0000,0.0,0.0
2020-06-24,0.0000,0.000000,0.0,0.0,0.000000,0.0000,0.0000,0.00,0.0,0.0000,...,0.0,0.0,0.0000,0.00000,0.0000,0.0000,0.00000,0.0000,0.0,0.0
2020-06-30,0.0000,0.000000,0.0,0.0,0.000000,0.0000,0.0000,0.00,0.0,0.0000,...,0.0,0.0,0.0000,0.00000,0.0000,0.0000,0.00000,0.0000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-07,0.5267,0.000000,0.0,0.0,0.084460,-0.2616,0.3350,0.00,0.0,0.1591,...,0.0,0.0,0.6486,0.21265,0.0000,0.0000,0.20095,0.0000,0.0,0.0
2021-10-08,0.3612,0.000000,0.0,0.0,0.086078,0.0000,0.0000,0.00,0.0,0.2500,...,0.0,0.0,0.0000,0.00000,0.4588,0.5859,0.00000,0.6486,0.0,0.0
2021-10-09,0.0000,0.000000,0.0,0.0,0.187640,0.0000,0.0000,0.00,0.0,0.0000,...,0.0,0.0,0.0000,0.00000,0.0000,0.0000,0.00000,0.0000,0.0,0.0


In [6]:
tickers_list[253] = "AMZN"
df5 = find_news(tickers_list[253:355])
df5 = df5.replace(np.nan, 0)

In [7]:
tickers_list[355] = "AMZN"
df6 = find_news(tickers_list[355:380])
df6 = df6.replace(np.nan, 0)

In [8]:
tickers_list[397] = "AMZN"
df7 = find_news(tickers_list[397:425])
df7 = df7.replace(np.nan, 0)

In [9]:
tickers_list[426] = "AMZN"
df8 = find_news(tickers_list[426:475])
df8 = df8.replace(np.nan, 0)

In [10]:
tickers_list[480] = "AMZN"
df9 = find_news(tickers_list[480:505])
df9 = df9.replace(np.nan, 0)

In [21]:
combined_sentiment_df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9], axis=1, sort=True) #, sort=True
combined_sentiment_df = combined_sentiment_df.replace(np.nan, 0)
combined_sentiment_df

ticker,AMZN,FB,MSFT,TSLA,AMZN,GOOG,GOOGL,A,ABBV,ABT,...,PBCT,PRGO,RL,SLG,UA,UAA,UNM,VNO,VNT,XRX
2019-10-28,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,...,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
2019-10-30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,...,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
2019-11-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,...,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
2019-11-11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,...,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
2019-11-12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,...,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-07,0.084460,-0.108231,0.120425,0.047263,0.084460,0.032014,0.032014,0.0,0.141243,0.3400,...,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-08,0.086078,0.131254,0.086670,0.070088,0.086078,0.026872,0.026872,0.0,0.636900,0.6369,...,0.4215,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-09,0.187640,0.248840,0.499050,0.049727,0.187640,-0.012487,-0.012487,0.0,0.000000,0.0000,...,0.0000,0.0,0.0,0.2584,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-10,0.190900,0.106200,0.000000,0.068233,0.190900,0.000167,0.000167,0.0,0.000000,0.0000,...,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
combined_sentiment_df.columns

Index(['AMZN', 'FB', 'MSFT', 'TSLA', 'AMZN', 'GOOG', 'GOOGL', 'A', 'ABBV',
       'ABT',
       ...
       'PBCT', 'PRGO', 'RL', 'SLG', 'UA', 'UAA', 'UNM', 'VNO', 'VNT', 'XRX'],
      dtype='object', name='ticker', length=482)

In [23]:
combined_sentiment_df.to_csv("Combined_Sentiment.csv")